# Load Mistral

In [ ]:
import os
import pandas as pd
import re

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
checkpoint = "mistralai/Mistral-7B-Instruct-v0.3"

model = AutoModelForCausalLM.from_pretrained(checkpoint, dtype=torch.bfloat16,
                                             attn_implementation="sdpa", device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

# Read data

In [ ]:
mount = '..'

In [ ]:
train = pd.read_json(f'{mount}/datasets/adept/train-dev-test-split/train.json')
dev = pd.read_json(f'{mount}/datasets/adept/train-dev-test-split/val.json')
test = pd.read_json(f'{mount}/datasets/adept/train-dev-test-split/test.json')

train['set'] = 'train'
dev['set'] = 'dev'
test['set'] = 'test'
df = pd.concat([train, dev, test])

In [ ]:
df.sample(5)

,sentence1,sentence2,modifier,noun,label,idx,set
12653,A hall is for concerts.,A historic hall is for concerts.,historic,hall,2,3252,train
7749,A friend shoulders a burden for a person.,A dead friend shoulders a burden for a person.,dead,friend,0,2162,train
9539,A tooth is for ripping.,An ordered tooth is for ripping.,ordered,tooth,0,13741,train
6956,Peritonitis may result from a burst mekels div...,Fungal peritonitis may result from a burst mek...,fungal,peritonitis,2,6270,train
4528,A wheel is used for holding a tire.,A fabricated wheel is used for holding a tire.,fabricated,wheel,3,1360,train


In [ ]:
df['set'].value_counts()

,count
set,
train,12892
test,1612
dev,1611


Only keep label classes 1, 2 and 3 (comparison labels). Map them to 0, 1 and 2 for training convenience:

* 1 => 0
* 2 => 1
* 3 => 2

In [ ]:
df = df[df['label'].isin([1, 2, 3])]
df['label'].value_counts()

,count
label,
2,10805
1,1868
3,1132


In [ ]:
df['label'] -= 1
df['label'].value_counts()

,count
label,
1,10805
0,1868
2,1132


Filter out duplicated data points

In [ ]:
df = df.drop_duplicates(subset=['sentence2'], keep='first')
df['label'].value_counts()

,count
label,
1,10800
0,1866
2,1129


In [ ]:
train = df[df['set'] == 'train']
dev = df[df['set'] == 'dev']
test = df[df['set'] == 'test']

In [ ]:
df['set'].value_counts()

,count
set,
train,11060
test,1380
dev,1355


# Trial run

In [ ]:
prompt_template = """Below are two sentences:
Sentence1: {}
Sentence2: {}
How likely is Sentence2 in relation to Sentence1? Respond with a number:
0 - Sentence2 is less likely than Sentence1
1 - Sentence2 is equally as likely as Sentence1
2 - Sentence2 is more likely than Sentence1
Respond only with the number. Do not include anything else in your response.
"""

'Below are two sentences:\nSentence1: You use a toy to play with a child.\nSentence2: You use a miraculous toy to play with a child.\nHow likely is Sentence2 in relation to Sentence1? Respond with a number:\n0 - Sentence2 is less likely than Sentence1\n1 - Sentence2 is equally as likely as Sentence1\n2 - Sentence2 is more likely than Sentence1\nRespond only with the number. Do not include anything else in your response.\n'

In [ ]:
s1 = 'You use a toy to play with a child.'
s2 = 'You use a miraculous toy to play with a child.'
prompt = prompt_template.format(s1, s2)
prompt

In [ ]:
messages = [
    {
        "role": "user",
        "content": prompt
    },
]

model_inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to(device)

generated_ids = model.generate(model_inputs,
                               max_new_tokens=10,
                               do_sample=True)
output = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

output

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'Below are two sentences:\nSentence1: You use a toy to play with a child.\nSentence2: You use a miraculous toy to play with a child.\nHow likely is Sentence2 in relation to Sentence1? Respond with a number:\n0 - Sentence2 is less likely than Sentence1\n1 - Sentence2 is equally as likely as Sentence1\n2 - Sentence2 is more likely than Sentence1\nRespond only with the number. Do not include anything else in your response.\n 0'

In [ ]:
output = output[len(prompt):].lstrip().rstrip()
output = int(output)
output

0

In [ ]:
# causes blabbering
inputs = tokenizer(prompt, return_tensors="pt")
generate_ids = model.generate(inputs.input_ids, max_new_tokens=10)
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'Below are two sentences:\nSentence1: You use a toy to play with a child.\nSentence2: You use a miraculous toy to play with a child.\nHow likely is Sentence2 in relation to Sentence1? Respond with a number:\n0 - Sentence2 is less likely than Sentence1\n1 - Sentence2 is equally as likely as Sentence1\n2 - Sentence2 is more likely than Sentence1\nRespond only with the number. Do not include anything else in your response.\n\n0\n\nSentence2 is less likely'

# Full run-through

In [ ]:
prompt_template = """Below are two sentences:
Sentence1: {}
Sentence2: {}
How likely is Sentence2 in relation to Sentence1? Respond with a number:
0 - Sentence2 is less likely than Sentence1
1 - Sentence2 is equally as likely as Sentence1
2 - Sentence2 is more likely than Sentence1
Respond only with the number. Do not include anything else in your response.
"""

In [ ]:
def ask_mistral(s1, s2):
  prompt = prompt_template.format(s1, s2)

  messages = [
    {
        "role": "user",
        "content": prompt
    },
  ]

  model_inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to(device)

  generated_ids = model.generate(model_inputs,
                                 pad_token_id=tokenizer.eos_token_id,
                                 max_new_tokens=10,
                                 do_sample=True)
  output = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

  output = output[len(prompt):].lstrip().rstrip()

  del prompt
  del messages
  del model_inputs
  del generated_ids

  return output

In [ ]:
from tqdm import tqdm
tqdm.pandas()

In [ ]:
# test set
test['mistral_raw'] = test.progress_apply(
    lambda row: ask_mistral(row['sentence1'], row['sentence2']),
    axis=1
    )
test.to_csv(f'{mount}/test_results/mistral_test.csv')

100%|██████████| 1380/1380 [43:22<00:00,  1.89s/it]
/tmp/ipython-input-1065309439.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['mistral_raw'] = test.progress_apply(


In [ ]:
# dev set
dev['mistral_raw'] = dev.progress_apply(
    lambda row: ask_mistral(row['sentence1'], row['sentence2']),
    axis=1
    )
dev.to_csv(f'{mount}/test_results/mistral_dev.csv')

100%|██████████| 1355/1355 [43:15<00:00,  1.92s/it]
/tmp/ipython-input-1146977946.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dev['mistral_raw'] = dev.progress_apply(


# Post-processing

In [ ]:
subset = 'test' # test or dev
results = pd.read_csv(f'{mount}/test_results/mistral_{subset}.csv', index_col=0)
results.head()

,sentence1,sentence2,modifier,noun,label,idx,set,mistral_raw
0,Urine is tested to check for disease.,Dark urine is tested to check for disease.,dark,urine,2,2082,test,0
1,News is relevant.,False news is relevant.,false,news,0,12641,test,0
2,Rice is stirred.,Fresh rice is stirred.,fresh,rice,1,11485,test,0
3,Plastic is made of carbon.,Colorful plastic is made of carbon.,colorful,plastic,1,886,test,0
4,Sometimes hiking causes a smile.,Sometimes abundant hiking causes a smile.,abundant,hiking,1,5863,test,1 - Sentence2 is equally as


In [ ]:
results['mistral_pred'] = results['mistral_raw'].apply(
    lambda x: int(x[0])
)
results['mistral_pred'].value_counts(dropna=False)

,count
mistral_pred,
0,951
1,411
2,18


In [ ]:
results['label'].value_counts(dropna=False)

,count
label,
1,1068
0,185
2,102


# Evaluation

## Standard

In [ ]:
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score

In [ ]:
def f1_one_vs_all(true, pred, class_label):
  true = [0 if v != class_label else 1 for v in true]
  pred = [0 if v != class_label else 1 for v in pred]
  return f1_score(true, pred)

In [ ]:
predictions = results['mistral_pred'].to_list()
labels = results['label'].to_list()

In [ ]:
macro_f1_test = f1_score(labels, predictions, average='macro')
print(f'{subset.capitalize()} set: macro F1 = {macro_f1_test:.3f}')

Dev set: macro F1 = 0.255


In [ ]:
f1_less, f1_eq, f1_more = f1_one_vs_all(labels, predictions, class_label=0), \
                          f1_one_vs_all(labels, predictions, class_label=1), \
                          f1_one_vs_all(labels, predictions, class_label=2)
f1_less, f1_eq, f1_more
print(f"""{subset.capitalize()} set: one-vs-all F1
1: {f1_less:.3f}
2: {f1_eq:.3f}
3: {f1_more:.3f}""")

Dev set: one-vs-all F1
1: 0.280
2: 0.432
3: 0.052


In [ ]:
acc_test = accuracy_score(labels, predictions)
print(f'{subset.capitalize()} set: accuracy = {acc_test:.3f}')

Dev set: accuracy = 0.353


## Cross-balanced

In [ ]:
from tqdm import tqdm

In [ ]:
def cross_balanced_eval(df, model_name, dev_or_test='test'):
    if dev_or_test == 'test':
        step = 101 # equal to the number of instances in the smallest class
    else:
        dev_or_test = 'dev'
        step = 102

    full_test_y_true = []
    full_test_y_pred = []
    all_macroF1 = 0
    all_label1_F1 = 0
    all_label2_F2 = 0
    all_label3_F3 = 0
    all_accuracy = 0
    iterations = 0

    for i in tqdm(range(
        1, len(df[df['set'] == dev_or_test].loc[(df['label'] == 1)]['label'].to_list())+1, step)):
        # get df slices containing 101 entries for each label
        new_df = df[df['set'] == dev_or_test].loc[(df['label'] == 0)][i:i+step]
        new_df = pd.concat([new_df, df[df['set'] == dev_or_test].loc[(df['label'] == 1)][i:i+step]])
        new_df = pd.concat([new_df, df[df['set'] == dev_or_test].loc[(df['label'] == 2)][i:i+step]])
        for i in [0, 1, 2]:
            if len(new_df.loc[(new_df['label'] == i)]['label'].to_list()) < step:
                wrap_around = step - len(new_df.loc[(new_df['label'] == i)]['label'].to_list())
                new_df = pd.concat([new_df, df[df['set'] == dev_or_test].loc[(df['label'] == i)][:wrap_around]])

        # evaluation
        test_set_predictions = new_df['mistral_pred'].to_list()
        test_labels = new_df['label'].to_list()

        full_test_y_true.extend(test_labels)
        full_test_y_pred.extend(test_set_predictions)

        macro_f1 = f1_score(test_labels, test_set_predictions, average='macro')
        all_macroF1 += macro_f1

        f1_less, f1_eq, f1_more = f1_one_vs_all(test_labels, test_set_predictions, class_label=0), \
                                f1_one_vs_all(test_labels, test_set_predictions, class_label=1), \
                                f1_one_vs_all(test_labels, test_set_predictions, class_label=2)
        all_label1_F1 += f1_less
        all_label2_F2 += f1_eq
        all_label3_F3 += f1_more

        accuracy = accuracy_score(test_labels, test_set_predictions)
        all_accuracy += accuracy

        iterations += 1

    print("")
    print(f"{model_name} - {dev_or_test} set: average stats")
    avr_MacroF1 = all_macroF1 / iterations
    print(f"macro F1: {avr_MacroF1:.3}")
    avr_accuracy = all_accuracy / iterations
    print(f'Accuracy: {avr_accuracy:.3f}')

    avr_label1_F1 = all_label1_F1 / iterations
    avr_label2_F2 = all_label2_F2 / iterations
    avr_label3_F3 = all_label3_F3 / iterations
    print(f'\nclass-wise F1 scores')
    print(f'1: {avr_label1_F1:.3f}\n2: {avr_label2_F2:.3f}\n3: {avr_label3_F3:.3f}')

    # create confusion matrix
    norm_setting = 'true'
    test_conf_matr = confusion_matrix(full_test_y_true, full_test_y_pred, normalize=norm_setting)
    test_conf_matr = pd.DataFrame(test_conf_matr, columns=['Less likely', 'Equally likely', 'More likely'],
                                index=['Less likely', 'Equally likely', 'More likely'])
    print("")
    print("True\\Predicted:")
    print(test_conf_matr)

In [ ]:
# dev
cross_balanced_eval(results, 'Mistral-7B-Instruct-v0.3', dev_or_test=subset)

100%|██████████| 11/11 [00:00<00:00, 46.85it/s]


Mistral-7B-Instruct-v0.3 - dev set: average stats
macro F1: 0.311
Accuracy: 0.394

class-wise F1 scores
1: 0.579
2: 0.299
3: 0.056

True\Predicted:
                Less likely  Equally likely  More likely
Less likely        0.853832        0.126560     0.019608
Equally likely     0.693405        0.299465     0.007130
More likely        0.401961        0.568627     0.029412


In [ ]:
# test
cross_balanced_eval(results, 'Mistral-7B-Instruct-v0.3', dev_or_test=subset)

100%|██████████| 11/11 [00:00<00:00, 34.68it/s]


Mistral-7B-Instruct-v0.3 - test set: average stats
macro F1: 0.342
Accuracy: 0.414

class-wise F1 scores
1: 0.569
2: 0.331
3: 0.126

True\Predicted:
                Less likely  Equally likely  More likely
Less likely        0.860486        0.119712     0.019802
Equally likely     0.680468        0.311431     0.008101
More likely        0.485149        0.445545     0.069307
